In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from numpy import random
import matplotlib.pyplot as plt
import seaborn as sns
from torch.utils.data import DataLoader
from collections import deque
import numpy as np
import multiprocessing
import time
from numpy import array
import pandas as pd

torch.manual_seed(1)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from sklearn.preprocessing import MinMaxScaler
#rotate a list by given offset, (negative offset, right to left) 
def rotateList(items, offset):
    return np.roll(items,offset)

def secondToMillisecond(val):
    return val*1000

def gaussiantimeseries(limit = 60, interval = 2):
    data = {}
    for t in range(0,limit,interval):
        data[t]=random.normal(loc = random.uniform(1,100), scale =1.5, size =1000).astype(float)
    return data

def createsignal(limit = 60, interval = 2):
    signal1, signal2, offset = [],[],[]
    limit = secondToMillisecond(limit)
    offset = random.randint(-10,10)
    data = gaussiantimeseries(limit, interval)
    for t in data:
        dist = data[t]
        signal1.append(random.choice(dist))
        signal2.append(random.choice(dist))
    return array(signal1).reshape((len(signal1), 1)), array(signal2).reshape((len(signal2), 1)), array(rotateList(signal2,offset)).reshape((len(signal2), 1)), offset

def createsignals(num = 50, limit = 60, interval = 2, normalize = False):
    input_seq = []
    data = []
    for i in range(num):
        sig1, sig2original, sig2, offset = createsignal(limit = limit)
        data.append([sig1, sig2original, sig2, offset])
        train_data = np.hstack((sig1,sig2)).astype(float)
        #train data without normalization
        if not normalize:
            train_data = torch.FloatTensor(train_data)
            input_seq.append((train_data, torch.FloatTensor([offset])))
        else:
            scaler = MinMaxScaler(feature_range=(-1, 1))
            train_data_normalized = scaler.fit_transform(train_data)
            train_data_normalized = torch.FloatTensor(train_data_normalized)
            input_seq.append((train_data_normalized, torch.FloatTensor([offset])))
    return input_seq, data
# print(createsignal())

In [ ]:
signal1,signal2,signal2rotated, offset = createsignal(limit=5, interval = 1)
print(f"signal1 {signal1.T}")
print(f"signal2 {signal2.T}")
print(f"signal2 offset {signal2rotated.T}")
print(f"offset {offset}")
# print(torch.FloatTensor(np.hstack((array(a),array(b)))).shape)

signal1 [[89.19262649  2.3358773   9.61857725 ... 84.84878357 30.380714
  44.24270055]]
signal2 [[91.48853034  1.20354769 11.92008974 ... 87.31665907 28.36057138
  43.17548394]]
signal2 offset [[80.44920154 44.95623505 12.26072353 ... 28.06016582 33.31552582
  58.73089043]]
offset 119


In [ ]:
# df = pd.DataFrame(columns=['signal1', 'signal2original', 'signal2', 'offset'])
_, data = createsignals(num=1, limit =5)
df = pd.DataFrame(data, columns=['signal1', 'signal2original', 'signal2', 'offset'])

In [ ]:
df

,signal1,signal2original,signal2,offset
0,"[[5.930678916974507], [38.49006620246865], [14...","[[6.035118328570036], [38.45475506634391], [17...","[[94.70916840860434], [61.96297207657408], [41...",9


In [ ]:
input = createsignals(num=1, limit=10, normalize=True)
print(input)

([(tensor([[ 0.4147,  0.8129],
        [-0.8651, -0.9308],
        [ 0.7457,  0.6830],
        ...,
        [ 0.1980,  0.5705],
        [-0.3005,  0.3524],
        [ 0.5812, -0.8763]]), tensor([-2.]))], [[array([[72.28869607],
       [ 4.93130804],
       [89.70816237],
       ...,
       [60.88171403],
       [34.64824228],
       [81.04813   ]]), array([[68.87377515],
       [ 5.28642633],
       [92.7059981 ],
       ...,
       [61.43999153],
       [34.19913141],
       [80.16177123]]), array([[92.7059981 ],
       [ 2.46214377],
       [85.97967683],
       ...,
       [80.16177123],
       [68.87377515],
       [ 5.28642633]]), -2]])


In [3]:
train_input, train_data = createsignals(num=5, limit = 50, interval = 1, normalize = False)
test_input, test_data = createsignals(num = 1)

In [ ]:
train_df = pd.DataFrame(train_data, columns=['signal1', 'signal2original', 'signal2', 'offset'])

In [ ]:
train_df

,signal1,signal2original,signal2,offset
0,"[[8.272737563939712], [64.04816055105094], [45...","[[8.033218702124838], [65.47197465272303], [46...","[[93.57843139238676], [18.834095779970216], [1...",8
1,"[[45.62819169117811], [70.1579610204642], [65....","[[47.0300842310897], [71.29131026608702], [65....","[[91.90151837652952], [47.0300842310897], [71....",1
2,"[[48.73042099173176], [28.94139230900702], [9....","[[51.78751991249547], [28.024456567417207], [7...","[[40.56029116372299], [33.31249929047154], [4....",-8
3,"[[9.18900852882664], [83.40387751741376], [78....","[[13.44528235004161], [82.61463029726661], [76...","[[75.34861057815009], [3.929373657690669], [15...",-8
4,"[[82.92269144408316], [55.77111913543482], [53...","[[82.41464552330662], [57.43215285363826], [53...","[[68.55908517446301], [61.92811923584845], [18...",-4


In [ ]:
train_input[:1]

[(tensor([[50.6323, 18.9472],
          [67.0676, 43.5327],
          [44.4008, 27.8539],
          ...,
          [29.5592, 70.8102],
          [ 1.5655, 71.4079],
          [16.6151, 63.4416]]), tensor([-160.]))]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!mkdir '/content/drive/My Drive/DummyGaussianData'


In [ ]:
train_df.to_csv (r'/content/drive/My Drive/DummyGaussianData/train_df.csv', index = False, header=True)

In [ ]:
dataframe = pd.read_csv('/content/drive/My Drive/DummyGaussianData/train_df.csv')

In [ ]:
dataframe

,signal1,signal2original,signal2,offset
0,[[ 8.27273756]\n [64.04816055]\n [45.71549104]...,[[ 8.0332187 ]\n [65.47197465]\n [46.20769999]...,[[93.57843139]\n [18.83409578]\n [15.33405303]...,8
1,[[45.62819169]\n [70.15796102]\n [65.85784427]...,[[47.03008423]\n [71.29131027]\n [65.83178518]...,[[91.90151838]\n [47.03008423]\n [71.29131027]...,1
2,[[48.73042099]\n [28.94139231]\n [ 9.04599211]...,[[51.78751991]\n [28.02445657]\n [ 7.48191239]...,[[40.56029116]\n [33.31249929]\n [ 4.38987107]...,-8
3,[[ 9.18900853]\n [83.40387752]\n [78.15877427]...,[[13.44528235]\n [82.6146303 ]\n [76.13504448]...,[[75.34861058]\n [ 3.92937366]\n [15.49013778]...,-8
4,[[82.92269144]\n [55.77111914]\n [53.79998206]...,[[82.41464552]\n [57.43215285]\n [53.35355593]...,[[ 68.55908517]\n [ 61.92811924]\n [ 18.031107...,-4


In [ ]:
    for col in train_df.columns:
        print(train_df[col])
        break

0    [[8.272737563939712], [64.04816055105094], [45...
1    [[45.62819169117811], [70.1579610204642], [65....
2    [[48.73042099173176], [28.94139230900702], [9....
3    [[9.18900852882664], [83.40387751741376], [78....
4    [[82.92269144408316], [55.77111913543482], [53...
Name: signal1, dtype: object


In [ ]:
dataframe.astype(float)

ValueError: ignored

In [ ]:
    for col in dataframe.columns:
        print(dataframe[col][0].replace('\n', ',')
        break

AttributeError: ignored

In [ ]:
def createsignalsfromdf(df, normalize = False):
    input_seq = []
    for col in df.columns:
        df[col] = df[col].replace('\n','').astype(float)
    for index, row in df.iterrows():
        sig1, sig2original, sig2, offset = row['signal1'], row['signal2original'], row['signal2'], row['offset']
        train_data = np.hstack((sig1,sig2)).astype(float)
        #train data without normalization
        if not normalize:
            train_data = torch.FloatTensor(train_data)
            input_seq.append((train_data, torch.FloatTensor([offset])))
        else:
            scaler = MinMaxScaler(feature_range=(-1, 1))
            train_data_normalized = scaler.fit_transform(train_data)
            train_data_normalized = torch.FloatTensor(train_data_normalized)
            input_seq.append((train_data_normalized, torch.FloatTensor([offset])))
    return input_seq

train_input = createsignalsfromdf(dataframe)

In [4]:
class LSTM(nn.Module):
    def __init__(self, input_size=1, hidden_layer_size=100,num_layers =2, output_size=1):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = nn.LSTM(input_size, hidden_layer_size)

        self.linear = nn.Linear(hidden_layer_size, output_size)

        self.hidden_cell = (torch.zeros(1,1,self.hidden_layer_size),
                            torch.zeros(1,1,self.hidden_layer_size))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq) ,1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return torch.round(predictions[-1])

In [5]:
import torch.backends.cudnn as cudnn
cudnn.benchmark = True
model = LSTM(input_size=2).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.5, amsgrad=False)

In [6]:
epochs = 50
model.train()
for epoch in range(epochs):
    print("Epoch {}/{}".format(epoch+1, epochs))
    print('-' * 10)
    epoch_loss = 0.0
    running_loss = 0.0
    model.zero_grad()
    for seq, labels in train_input:

        model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
                        torch.zeros(1, 1, model.hidden_layer_size)).to(device)
        # kseq, labels = seq.to(device), labels.to(device)
        y_pred = model(seq)
        # y_pred = y_pred.to(device)
        # print("here")
        optimizer.zero_grad()
        loss = criterion(y_pred, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += y_pred.shape[0] * loss.item()

        # print statistics
        # running_loss += loss.item()
        # if i % 25 == 1:    
        #     print('[%d] loss: %.3f' %
        #           (i + 1, running_loss / 25))
        #     running_loss = 0.0
    print("Epoch {} Loss {}".format(epoch+1, epoch_loss / len(train_input)))

    # if i%25 == 1:
#     print(f'epoch: {i:3} loss: {loss.item():10.8f}')

# print(f'epoch: {i:3} loss: {loss.item():10.10f}')

Epoch 1/50
----------


AttributeError: ignored

In [ ]:
epochs = 1
model.eval()
for epoch in range(epochs):
    print("Epoch {}/{}".format(epoch+1, epochs))
    print('-' * 10)
    epoch_loss = 0.0
    running_loss = 0.0
    # model.zero_grad()
    for seq, labels in test_input:

        # model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
        #                 torch.zeros(1, 1, model.hidden_layer_size))
        # # seq, labels = seq.to(device), labels.to(device)
        y_pred = model(seq)#.to(device)
        print(f"Offset {labels.data[0]}, Prediction {y_pred.data[0]}")
        # optimizer.zero_grad()
        loss = criterion(y_pred, labels)
        # loss.backward()
        # optimizer.step()
        epoch_loss += y_pred.shape[0] * loss.item()

        # print statistics
        # running_loss += loss.item()
        # if i % 25 == 1:    
        #     print('[%d] loss: %.3f' %
        #           (i + 1, running_loss / 25))
        #     running_loss = 0.0
    print("Epoch {} Loss {}".format(epoch+1, epoch_loss / len(train_input)))

    # if i%25 == 1:
#     pr

Epoch 1/1
----------
Offset -1.0, Prediction -1.3300405740737915
Offset 1.0, Prediction 1.00392484664917
Offset 3.0, Prediction 2.5719196796417236
Offset -3.0, Prediction -3.5593316555023193
Offset 3.0, Prediction -1.3332680463790894
Offset -1.0, Prediction -0.802498996257782
Offset -5.0, Prediction -2.7559022903442383
Offset -4.0, Prediction -1.5229063034057617
Offset 0.0, Prediction 0.006116407923400402
Offset 0.0, Prediction 0.018826615065336227
Offset -4.0, Prediction -3.026475667953491
Offset 0.0, Prediction -0.30811968445777893
Offset -5.0, Prediction -4.173523902893066
Offset 2.0, Prediction 1.4957826137542725
Offset 1.0, Prediction 1.166930079460144
Offset -1.0, Prediction -0.7062098383903503
Offset 1.0, Prediction 1.3860684633255005
Offset 0.0, Prediction -0.16058781743049622
Offset 2.0, Prediction 1.7859256267547607
Offset 2.0, Prediction 1.931938886642456
Offset -5.0, Prediction -4.232842922210693
Offset 4.0, Prediction 2.8655223846435547
Offset 2.0, Prediction 2.14780688285